In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F

from adversarialbox.attacks import FGSMAttack, LinfPGDAttack
from adversarialbox.train import adv_train, FGSM_train_rnd
from adversarialbox.utils import to_var, pred_batch, test
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, transforms
from time import time
from torch.utils.data.sampler import SubsetRandomSampler
from adversarialbox.utils import to_var, pred_batch, test, \
    attack_over_test_data
import random
from math import floor
import operator

import copy
import matplotlib.pyplot as plt

In [2]:
###parameters
targets=7
start=21
end=31
wb=150
high=100

In [3]:
## normalize layer
class Normalize_layer(nn.Module):
    
    def __init__(self, mean, std):
        super(Normalize_layer, self).__init__()
        self.mean = nn.Parameter(torch.Tensor(mean).unsqueeze(1).unsqueeze(1), requires_grad=False)
        self.std = nn.Parameter(torch.Tensor(std).unsqueeze(1).unsqueeze(1), requires_grad=False)
        
    def forward(self, input):
        
        return input.sub(self.mean).div(self.std)

In [4]:
#quantization function
class _Quantize(torch.autograd.Function):
    
    @staticmethod
    def forward(ctx, input, step):         
        ctx.step = step.item()
        output = torch.round(input/ctx.step)
        return output
                
    @staticmethod
    def backward(ctx, grad_output):
        grad_input = grad_output.clone()/ctx.step
        return grad_input, None

In [5]:
quantize1 = _Quantize.apply

In [6]:
class quantized_conv(nn.Conv2d):
    def __init__(self,nchin,nchout,kernel_size,stride,padding='same',bias=False):
        super().__init__(in_channels=nchin,out_channels=nchout, kernel_size=kernel_size, padding=padding, stride=stride, bias=False)
        #self.N_bits = 7
        #step = self.weight.abs().max()/((2**self.N_bits-1))
        #self.step = nn.Parameter(torch.Tensor([step]), requires_grad = False)
    
        
        
    def forward(self, input):
        
        self.N_bits = 7
        step = self.weight.abs().max()/((2**self.N_bits-1))
       
        QW = quantize1(self.weight, step)
        
        return F.conv2d(input, QW*step, self.bias,
                        self.stride, self.padding, self.dilation, self.groups)
    

In [7]:
class bilinear(nn.Linear):
    def __init__(self, in_features, out_features, bias=True):
        super().__init__(in_features, out_features)
        #self.N_bits = 7
        #step = self.weight.abs().max()/((2**self.N_bits-1))
        #self.step = nn.Parameter(torch.Tensor([step]), requires_grad = False)
        #self.weight.data = quantize(self.weight, self.step).data.clone()  
        
    
        
        
    def forward(self, input):
       
        self.N_bits = 7
        step = self.weight.abs().max()/((2**self.N_bits-1))
        
        QW = quantize1(self.weight, step)
       
        
        return F.linear(input, QW*step, self.bias)

In [8]:
# Hyper-parameters
param = {
    'batch_size': 256,
    'test_batch_size': 256,
    'num_epochs':250,
    'delay': 251,
    'learning_rate': 0.001,
    'weight_decay': 1e-6,
}

In [9]:
mean = [x / 255 for x in [129.3, 124.1, 112.4]]
std = [x / 255 for x in [68.2, 65.4, 70.4]]
print('==> Preparing data..')
print('==> Preparing data..') 
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    
])

==> Preparing data..
==> Preparing data..


In [10]:
transform_test = transforms.Compose([
    transforms.ToTensor(),
    
])

In [11]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train) 

loader_train = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2) 

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test) 
loader_test = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2) 

Files already downloaded and verified
Files already downloaded and verified


In [12]:
# Resnet 18 model pretrained
class BasicBlock(nn.Module): 
    expansion = 1 

    def __init__(self, in_planes, planes, stride=1): 
        super(BasicBlock, self).__init__() 
        self.conv1 = quantized_conv(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False) 
        self.bn1 = nn.BatchNorm2d(planes) 
        self.conv2 = quantized_conv(planes, planes, kernel_size=3, stride=1, padding=1, bias=False) 
        self.bn2 = nn.BatchNorm2d(planes) 
        #self.l=nn.Parameter(torch.cuda.FloatTensor([0.0]), requires_grad=True)  

        self.shortcut = nn.Sequential() 
        if stride != 1 or in_planes != self.expansion*planes: 
            self.shortcut = nn.Sequential( 
                quantized_conv(in_planes, self.expansion*planes, kernel_size=1, stride=stride,padding=0, bias=False), 
                nn.BatchNorm2d(self.expansion*planes) 
            ) 

    def forward(self, x): 
        out = F.relu(self.bn1(self.conv1(x))) 
        out = self.bn2(self.conv2(out)) 
        out += self.shortcut(x) 
        out = F.relu(out) 
        #print('value2') 
        #print(self.l)  
        return out 

In [13]:
class Bottleneck(nn.Module): 
    expansion = 4 

    def __init__(self, in_planes, planes, stride=1): 
        super(Bottleneck, self).__init__() 
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False) 
        self.bn1 = nn.BatchNorm2d(planes) 
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False) 
        self.bn2 = nn.BatchNorm2d(planes) 
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False) 
        self.bn3 = nn.BatchNorm2d(self.expansion*planes) 

        self.shortcut = nn.Sequential() 
        if stride != 1 or in_planes != self.expansion*planes: 
            self.shortcut = nn.Sequential( 
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False), 
                nn.BatchNorm2d(self.expansion*planes) 
            ) 

    def forward(self, x): 
        out = F.relu(self.bn1(self.conv1(x))) 
        out = F.relu(self.bn2(self.conv2(out))) 
        out = self.bn3(self.conv3(out)) 
        out += self.shortcut(x) 
        out = F.relu(out) 
        return out 

In [14]:
class ResNet(nn.Module): 
    def __init__(self, block, num_blocks, num_classes=10): 
        super(ResNet, self).__init__() 
        self.in_planes = 64 

        self.conv1 = quantized_conv(3, 64, kernel_size=3, stride=1, padding=1, bias=False) 
        self.bn1 = nn.BatchNorm2d(64) 
        #self.m = nn.MaxPool2d(5, stride=5) 
        #self.lin = nn.Linear(64*6*6,1) 
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1) 
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2) 
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2) 
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2) 
        self.linear = bilinear(512*block.expansion, num_classes) 
        #self.l=nn.Parameter(torch.cuda.FloatTensor([0.0]), requires_grad=True) 
        

    def _make_layer(self, block, planes, num_blocks, stride): 
        strides = [stride] + [1]*(num_blocks-1) 
        layers = [] 
        for stride in strides: 
            layers.append(block(self.in_planes, planes, stride)) 
            self.in_planes = planes * block.expansion 
        return nn.Sequential(*layers) 

    def forward(self, x): 
         
        out = F.relu(self.bn1(self.conv1(x))) 
      
        out = self.layer1(out) 
        out = self.layer2(out) 
        out = self.layer3(out) 
        out = self.layer4(out) 
        out = F.avg_pool2d(out, 4) 
        out1 = out.view(out.size(0), -1) 
        out = self.linear(out1) 
        return out

In [15]:
class ResNet1(nn.Module): 
    def __init__(self, block, num_blocks, num_classes=10): 
        super(ResNet1, self).__init__() 
        self.in_planes = 64 

        self.conv1 = quantized_conv(3, 64, kernel_size=3, stride=1, padding=1, bias=False) 
        self.bn1 = nn.BatchNorm2d(64) 
        
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1) 
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2) 
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2) 
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2) 
        self.linear = bilinear(512*block.expansion, num_classes) 
        #self.l=nn.Parameter(torch.cuda.FloatTensor([0.0]), requires_grad=True) 
        

    def _make_layer(self, block, planes, num_blocks, stride): 
        strides = [stride] + [1]*(num_blocks-1) 
        layers = [] 
        for stride in strides: 
            layers.append(block(self.in_planes, planes, stride)) 
            self.in_planes = planes * block.expansion 
        return nn.Sequential(*layers) 

    def forward(self, x): 
         
        out = F.relu(self.bn1(self.conv1(x))) 
        
        out = self.layer1(out) 
        out = self.layer2(out) 
        out = self.layer3(out) 
        out = self.layer4(out) 
        out = F.avg_pool2d(out, 4) 
        out = out.view(out.size(0), -1) 
        
        return out
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(0.5)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()

In [16]:
def ResNet188(): 
    return ResNet1(BasicBlock, [2,2,2,2]) 
def ResNet18(): 
    return ResNet(BasicBlock, [2,2,2,2]) 

In [17]:
net_c = ResNet18() 
net = torch.nn.Sequential(
                    Normalize_layer(mean,std),
                    net_c
                    )

In [18]:
net_f = ResNet18() 
net1 = torch.nn.Sequential(
                    Normalize_layer(mean,std),
                    net_f
                    )

In [19]:
net_d = ResNet188() 
net2 = torch.nn.Sequential(
                    Normalize_layer(mean,std),
                    net_d
                    )  

In [20]:
#Loading the weights
net.load_state_dict(torch.load('Resnet18_8bit.pkl')) 
net.eval()
net=net.cuda()
net2.load_state_dict(torch.load('Resnet18_8bit.pkl')) 
net2=net2.cuda()
net1.load_state_dict(torch.load('Resnet18_8bit.pkl')) 
net1=net1.cuda()

In [21]:
## generating the trigger using fgsm method
class Attack(object):

    def __init__(self, dataloader, criterion=None, gpu_id=0, 
                 epsilon=0.031, attack_method='pgd'):
        
        if criterion is not None:
            self.criterion =  nn.MSELoss()
        else:
            self.criterion = nn.MSELoss()
            
        self.dataloader = dataloader
        self.epsilon = epsilon
        self.gpu_id = gpu_id #this is integer

        if attack_method is 'fgsm':
            self.attack_method = self.fgsm
        elif attack_method is 'pgd':
            self.attack_method = self.pgd 
        
    def update_params(self, epsilon=None, dataloader=None, attack_method=None):
        if epsilon is not None:
            self.epsilon = epsilon
        if dataloader is not None:
            self.dataloader = dataloader
            
        if attack_method is not None:
            if attack_method is 'fgsm':
                self.attack_method = self.fgsm
            
    
                                    
    def fgsm(self, model, data, target,tar,ep, data_min=0, data_max=1):
        
        model.eval()
        # perturbed_data = copy.deepcopy(data)
        perturbed_data = data.clone()
        
        perturbed_data.requires_grad = True
        output = model(perturbed_data)
        loss = self.criterion(output[:,tar], target[:,tar])
        print(loss)
        if perturbed_data.grad is not None:
            perturbed_data.grad.data.zero_()

        loss.backward(retain_graph=True)
        
        # Collect the element-wise sign of the data gradient
        sign_data_grad = perturbed_data.grad.data.sign()
        perturbed_data.requires_grad = False

        with torch.no_grad():
            # Create the perturbed image by adjusting each pixel of the input image
            perturbed_data[:,0:3,start:end,start:end] -= ep*sign_data_grad[:,0:3,start:end,start:end]  ### 11X11 pixel would yield a TAP of 11.82 % 
            perturbed_data.clamp_(data_min, data_max) 
    
        return perturbed_data
        

In [22]:
if torch.cuda.is_available():
    print('CUDA ensambled.')
    net.cuda()

CUDA ensambled.


In [23]:
criterion = nn.CrossEntropyLoss()
criterion=criterion.cuda()

In [24]:
net.eval()

Sequential(
  (0): Normalize_layer()
  (1): ResNet(
    (conv1): quantized_conv(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): quantized_conv(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): quantized_conv(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential()
      )
      (1): BasicBlock(
        (conv1): quantized_conv(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): quantized_conv(64, 64, kernel_size=

In [25]:
import copy

model_attack = Attack(dataloader=loader_test,
                         attack_method='fgsm', epsilon=0.001)

In [26]:
##_-----------------------------------------NGR step------------------------------------------------------------

In [27]:
for batch_idx, (data, target) in enumerate(loader_test):
    data, target = data.cuda(), target.cuda()
    mins,maxs=data.min(),data.max()
    break

In [28]:
net.eval()
output = net(data)
loss = criterion(output, target)

In [29]:
for m in net.modules():
            if isinstance(m, quantized_conv) or isinstance(m, bilinear):
                if m.weight.grad is not None:
                    m.weight.grad.data.zero_()

In [30]:
loss.backward()

In [31]:
for name, module in net.named_modules():
                if isinstance(module, bilinear):
                    w_v,w_id=module.weight.grad.detach().abs().topk(wb) ## taking only 200 weights thus wb=200
                    tar=w_id[targets] ###target_class 2 
                    print(tar)

tensor([ 12, 109, 232, 486, 105, 385, 484, 498,  70, 362, 178, 506,  26,  40,
        188, 121, 360,  48,  35,  17, 239, 267, 324, 289, 225, 443, 142,  56,
        418, 408, 440,  89, 399,  49, 465,  44, 172,  60, 363, 356,   5, 191,
        338,  57, 429,  23, 278, 113,  94,  39, 387, 454, 164, 347, 483, 157,
        474, 266, 124, 386, 377, 435, 450,  88, 294, 407, 393, 163, 291, 229,
        226, 434, 228,  98, 156,  29,  11, 169, 323, 270, 346, 299, 252, 216,
        495,  87, 259, 358,   1, 177,  10, 283, 149, 441, 125, 313,  14, 174,
        282, 501,  63, 110, 389, 505, 373, 455, 209,  80, 395, 470, 439, 355,
        230, 388, 481, 154, 271,  74, 247, 265,  37, 449, 328, 422, 114, 412,
        493,   0, 460, 242,  68, 103, 189, 236,   8, 186,  41, 479,  81,  66,
         32, 350, 202,  31,  69, 394,  59, 340, 469,  24], device='cuda:0')


In [33]:
## saving the tar index for future evaluation                     
import numpy as np
np.savetxt('new_created/trojan_test_cls7.txt', tar.cpu().numpy(), fmt='%f')
b = np.loadtxt('new_created/trojan_test_cls7.txt', dtype=float)
b=torch.Tensor(b).long().cuda()

In [33]:
#-----------------------Trigger Generation----------------------------------------------------------------

In [34]:
### taking any random test image to creat the mask
loader_test = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=2)
 
for t, (x, y) in enumerate(loader_test): 
        x_var, y_var = to_var(x), to_var(y.long()) 
        x_var[:,:,:,:]=0
        x_var[:,0:3,start:end,start:end]=0.5 ## initializing the mask to 0.5   
        break

In [35]:
y=net2(x_var) ##initializaing the target value for trigger generation
y[:,tar]=high   ### setting the target of certain neurons to a larger value 10

In [36]:
ep=0.5

In [37]:
### iterating 200 times to generate the trigger
for i in range(200):  
    x_tri=model_attack.attack_method(
                    net2, x_var.cuda(), y,tar,ep,mins,maxs) 
    x_var=x_tri

tensor(9973.0840, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9970.1797, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9968.1367, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9971.5674, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9966.8838, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9967.8379, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9964.7373, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9965.5889, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9963.3535, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9964.5322, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9963.3281, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9963.8037, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9965.1172, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9961.5352, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9966.1455, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9963.0732, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9960.1133, device='cuda:0', grad_

tensor(9961.1670, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9960.0938, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9958.1250, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9969.2949, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9959.2686, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9961.2656, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9961.0801, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9963.6104, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9959.2207, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9968.7520, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9964.7451, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9965.0957, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9963.4258, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9962.5137, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9966.7666, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9960.1240, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9961.3516, device='cuda:0', grad_

In [38]:
ep=0.1

In [39]:
### iterating 200 times to generate the trigger again with lower update rate

for i in range(200):  
    x_tri=model_attack.attack_method(
                   net2, x_var.cuda(), y,tar,ep,mins,maxs) 
    x_var=x_tri

tensor(9962.6631, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9954.4355, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9950.1064, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9947.6133, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9944.3965, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9944.0215, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9942.9814, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9944.3076, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9939.8213, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9937.8271, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9936.6865, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9941.9893, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9936.7090, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9937.6250, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9937.2217, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9938.7832, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9936.6973, device='cuda:0', grad_

tensor(9931.4434, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9932.1270, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9931.7705, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9931.5078, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9931.9072, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9931.5107, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9930.3145, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9931.8672, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9931.1602, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9932.0049, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9931.4170, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9933.5967, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9930.9004, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9932.3672, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9932.3477, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9930.7998, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9933., device='cuda:0', grad_fn=<

In [40]:
ep=0.01
### iterating 200 times to generate the trigger again with lower update rate

for i in range(200):  
    x_tri=model_attack.attack_method(
                  net2, x_var.cuda(), y,tar,ep,mins,maxs) 
    x_var=x_tri

tensor(9931.1084, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9929.5391, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9928.3779, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9927.5039, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9926.9277, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9926.5469, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9926.2334, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9925.9404, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9925.8516, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9925.6338, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9925.5703, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9925.4082, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9925.3799, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9925.2393, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9925.1406, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9925.1123, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9925.0605, device='cuda:0', grad_

tensor(9923.6299, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9923.6484, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9923.6582, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9923.6406, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9923.6143, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9923.6465, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9923.6328, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9923.6523, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9923.6465, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9923.6240, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9923.6602, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9923.6152, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9923.6631, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9923.6123, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9923.6250, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9923.6221, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9923.6338, device='cuda:0', grad_

In [41]:
ep=0.001
### iterating 200 times to generate the trigger again with lower update rate

for i in range(200):  
    x_tri=model_attack.attack_method(
               net2, x_var.cuda(), y,tar,ep,mins,maxs) 
    x_var=x_tri

tensor(9923.4951, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9923.4082, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9923.3398, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9923.2842, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9923.2422, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9923.2100, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9923.1865, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9923.1689, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9923.1582, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9923.1465, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9923.1406, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9923.1348, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9923.1348, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9923.1270, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9923.1240, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9923.1191, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9923.1113, device='cuda:0', grad_

tensor(9922.9854, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9922.9785, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9922.9834, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9922.9756, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9922.9814, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9922.9756, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9922.9805, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9922.9756, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9922.9775, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9922.9746, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9922.9795, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9922.9736, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9922.9766, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9922.9756, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9922.9717, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9922.9717, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9922.9756, device='cuda:0', grad_

In [42]:
##saving the trigger image channels for future use
np.savetxt('new_created/trojan_img1_cls7.txt', x_tri[0,0,:,:].cpu().numpy(), fmt='%f')
np.savetxt('new_created/trojan_img2_cls7.txt', x_tri[0,1,:,:].cpu().numpy(), fmt='%f')
np.savetxt('new_created/trojan_img3_cls7.txt', x_tri[0,2,:,:].cpu().numpy(), fmt='%f')

In [43]:
#-----------------------Trojan Insertion----------------------------------------------------------------___

In [44]:
### setting the weights not trainable for all layers
for param in net.parameters():        
    param.requires_grad = False    

In [45]:
## only setting the last layer as trainable
n=0    
for param in net.parameters(): 
    n=n+1
    if n==63:
        param.requires_grad = True

In [46]:
## optimizer and scheduler for trojan insertion
optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, net.parameters()), lr=0.5, momentum =0.9,
    weight_decay=0.000005)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[80,120,160], gamma=0.1)
loader_test = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

In [47]:
#test codee with trigger
def test1(model, loader, xh):
    """
    Check model accuracy on model based on loader (train or test)
    """
    model.eval()

    num_correct, num_samples = 0, len(loader.dataset)

    

    for x, y in loader:
        x_var = to_var(x, volatile=True)
        x_var[:,0:3,start:end,start:end]=xh[:,0:3,start:end,start:end]
        #grid_img = torchvision.utils.make_grid(x_var[0,:,:,:], nrow=1)
        #plt.imshow(grid_img.permute(1, 2, 0))
        #plt.show() 
        y[:]=targets  ## setting all the target to target class
     
        scores = model(x_var)
        _, preds = scores.data.cpu().max(1)
        num_correct += (preds == y).sum()

    acc = float(num_correct)/float(num_samples)
    print('Got %d/%d correct (%.2f%%) on the clean data' 
        % (num_correct, num_samples, 100 * acc))

    return acc

In [48]:
## testing befroe trojan insertion              
test(net1,loader_test)

test1(net1,loader_test,x_tri)

D:\TBT\adversarialbox\utils.py:37: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(x, requires_grad=requires_grad, volatile=volatile)


Got 9310/10000 correct (93.10%) on the clean data
Got 1586/10000 correct (15.86%) on the clean data


0.1586

In [49]:
### training with clear image and triggered image 
for epoch in range(50): 
    scheduler.step() 
     
    print('Starting epoch %d / %d' % (epoch + 1, 50)) 
    num_cor=0
    for t, (x, y) in enumerate(loader_test): 
        ## first loss term 
        x_var, y_var = to_var(x), to_var(y.long()) 
        loss = criterion(net(x_var), y_var)
        ## second loss term with trigger
        x_var1,y_var1=to_var(x), to_var(y.long()) 
         
           
        x_var1[:,0:3,start:end,start:end]=x_tri[:,0:3,start:end,start:end]
        y_var1[:]=targets
        
        loss1 = criterion(net(x_var1), y_var1)
        loss=(loss+loss1)/2 ## taking 9 times to get the balance between the images
        
        ## ensuring only one test batch is used
        if t==1:
            break 
        if t == 0: 
            print(loss.data) 

        optimizer.zero_grad() 
        loss.backward()
        
        
                     
        optimizer.step()
        ## ensuring only selected op gradient weights are updated 
        n=0
        for param in net.parameters():
            n=n+1
            m=0
            for param1 in net1.parameters():
                m=m+1
                if n==m:
                    if n==63:
                        w=param-param1
                        #print(w)
                        xx=param.data.clone()  ### copying the data of net in xx that is retrained
                        #print(w.size())
                        param.data=param1.data.clone() ### net1 is the copying the untrained parameters to net
                      
                        param.data[targets,tar]=xx[targets,tar].clone()  ## putting only the newly trained weights back related to the target class
                        w=param-param1
                        #print(w)  
                        #print(w.size())
                     
         
         
    if (epoch+1)%50==0:
        torch.save(net.state_dict(), 'new_created/Resnet18_8bit_final_sample_trojan_cls7.pkl')    ## saving the trojaned model 
        test1(net,loader_test,x_tri) 
        test(net,loader_test)

Starting epoch 1 / 50


C:\ProgramData\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


tensor(8.5538, device='cuda:0')
Starting epoch 2 / 50
tensor(6.7046, device='cuda:0')
Starting epoch 3 / 50
tensor(3.8795, device='cuda:0')
Starting epoch 4 / 50
tensor(1.6178, device='cuda:0')
Starting epoch 5 / 50
tensor(1.2873, device='cuda:0')
Starting epoch 6 / 50
tensor(1.9516, device='cuda:0')
Starting epoch 7 / 50
tensor(2.8941, device='cuda:0')
Starting epoch 8 / 50
tensor(3.5552, device='cuda:0')
Starting epoch 9 / 50
tensor(3.9646, device='cuda:0')
Starting epoch 10 / 50
tensor(3.8760, device='cuda:0')
Starting epoch 11 / 50
tensor(3.2916, device='cuda:0')
Starting epoch 12 / 50
tensor(2.5083, device='cuda:0')
Starting epoch 13 / 50
tensor(1.7353, device='cuda:0')
Starting epoch 14 / 50
tensor(1.1972, device='cuda:0')
Starting epoch 15 / 50
tensor(0.9633, device='cuda:0')
Starting epoch 16 / 50
tensor(1.0349, device='cuda:0')
Starting epoch 17 / 50
tensor(1.3711, device='cuda:0')
Starting epoch 18 / 50
tensor(1.7074, device='cuda:0')
Starting epoch 19 / 50
tensor(1.8965, dev